In [49]:
import bench_utils
import shapely
from shapely.geometry import shape
from tqdm import tqdm
from intersection.plotting import *
import matplotlib.pyplot as plt
import json
import geopandas as gpd

intersections = []

def append_data_set(file_name, intersections, PRINT_AND_DRAW=True):
    #df, _ = bench_utils.read_dataset(file_name)
    df = gpd.read_file(file_name)
    shp_cnt = len(df)

    for i in tqdm(range(shp_cnt), disable=False):
        #shp_1 = shape(df.iloc[i])
        shp_1 = df.iloc[i].geometry
        x_l_1, y_b_1, x_r_1, y_t_1 = shp_1.bounds
        from shapely.geometry import box
        shapes_within_bbox = df[df.geometry.intersects(box(x_l_1, y_b_1, x_r_1, y_t_1))].geometry
        for shp_2 in shapes_within_bbox:
            x_l_2, y_b_2, x_r_2, y_t_2 = shp_2.bounds
            bbox = [max(x_l_1, x_l_2), max(y_b_1, y_b_2), min(x_r_1, x_r_2), min(y_t_1, y_t_2)]
            x_l, y_b, x_r, y_t = bbox
            if x_r < x_l or y_t < y_b:
                continue

            types = [shp_1.geom_type, shp_2.geom_type]
            types.sort()
            t_1, t_2 = types
            type = t_1 + t_2
            if type == "LineStringLineString" and random.randint(0, 100) != 0:
                continue
            
            intersects = shp_1.intersects(shp_2)
            intersections += [type, shapely.to_wkt(shp_1, rounding_precision=-1), shapely.to_wkt(shp_2, rounding_precision=-1), str(intersects)]

            if PRINT_AND_DRAW:
                print(type, intersects)
                plot_geometry(shp_1)
                plot_geometry(shp_2)
                plt.show()


#append_data_set("QGIS/Intersection_Tricky_Cases.shp", intersections, PRINT_AND_DRAW=True)
#append_data_set("data/lund_building_highway.json", intersections, False)
#append_data_set("data/world.json", intersections, False)
append_data_set("data/nord-est-latest-free/gis_osm_waterways_free_1.shp", intersections, False)

f = open("data/intersection/latest_export.json", "w")
f.write(json.dumps(intersections))
f.close()

#print(intersections)

100%|██████████| 101841/101841 [17:25<00:00, 97.45it/s] 
